# Importazione librerie + Caricamento datasets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
df= pd.read_csv(r'/home/omori/Scrivania/its/deep_learning_venv/dataset_calcio/SerieA23.csv')
dfbundes = pd.read_csv(r'/home/omori/Scrivania/its/deep_learning_venv/dataset_calcio/Bundesliga23.csv')
dfliga = pd.read_csv(r'/home/omori/Scrivania/its/deep_learning_venv/dataset_calcio/LaLiga23.csv')
dfprem = pd.read_csv(r'/home/omori/Scrivania/its/deep_learning_venv/dataset_calcio/PL23.csv')


df_completo = pd.concat([df,dfbundes,dfliga,dfprem],ignore_index=True)

In [ ]:
df.info()

In [ ]:
df
df_completo

# Preprocessing dei dati

In [ ]:
df_completo['FTR_num'] = df_completo['FTR'].map({'H':0,'D':1,'A':2}) #mappatura dei risultati utili

features = [ #seleziono solo le features che mi interessano per decidere una partita
    'HS',#Home shots    
    'AS',#Away shots
    'HST',#Home shots on target
    'AST',#Away shots on target
    'AF',#Away fouls
    'HY',#Home YELLOW cards
    'AY',#Away YELLOW cards
    'HR',#Home RED cards
    'AR',#Away RED cards
    'HC',#Home corners
    'AC',#Away corners
    'HF'#Home fouls
]

In [ ]:
X = df_completo[features]
y = df_completo['FTR_num']  #il target è il FULL TIME RESULT

scaler = StandardScaler() #standardizzo i dati
X_scaled = scaler.fit_transform(X)

#splitting dei dati 
X_train, X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=42)

# Rete Neurale

In [ ]:
modello = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)), #creo un layer dense con 64 neuroni, attivazione relu
    Dropout(0.3), #questo parametro "azzera i valori di alcuni neuroni" di volta in volta, per evitare overfitting
    Dense(32,activation='relu'),
    Dropout(0.2), 
    Dense(3,activation='softmax')
])

modello.compile(optimizer=Adam(learning_rate=0.0001), #ottimizzazione Adam, con learning rate bassissimo per permettere alla rete di non andare ad adattarsi tropoo
                  loss='sparse_categorical_crossentropy', #compatibilità molto buona con softmax
                  metrics=['accuracy']) #per l'accuracy del modello

In [ ]:
#addestramento 

hs = modello.fit(X_train,y_train,
                 epochs=500,
                 batch_size=32, #in quanti "blocchi" vengono passati i dati
                 validation_data=(X_test,y_test))

# Valutazione modello

In [ ]:
loss_test, accuracy_test = modello.evaluate(X_test,y_test)

print(f'Accuracy modello : {accuracy_test:.4f}')

# Simulazione di una partita 

In [121]:

#dati di una partita in ingresso

match_example = np.array([[14 #Home shots
                           ,8 #Away shots
                           ,3 #Home shots on target
                           ,13 #Away shots on target
                           ,12 #Away fouls 
                           ,2 #Home YELLOW cards
                           ,3 #Away YELLOW cards
                           ,2 #Home RED cards
                           ,4 #Away RED cards
                           ,6 #Home corners
                           ,4 #Away corners
                           ,14 #Home fouls
                           ]]) #passo secondo la posizione delle features i numeri delle statistiche di una partita

#features = [
#   'HS','AS','HST','AST',
#  'AF','HY','AY',
#    'HR','AR','HC','AC','HF'
#]




# Predizione risultato finale

In [123]:
match_ex_scaled = scaler.transform(match_example) #normalizzo i dati del nuovo match
prediction = modello.predict(match_ex_scaled) 

prob_vittoriaH = prediction[0][0]
prob_pareggio = prediction[0][1]
prob_vittoriaAW = prediction[0][2]




risultato = ['Vittoria Home','Pareggio','Vittoria Away'][np.argmax(prediction)]
print(f'Predizione risultato : {risultato}\ncon probabilità di Pareggio : {prob_pareggio:.1%},\nprobabilità di vittoria Casa : {prob_vittoriaH:.1%}\ne probabilità di vittoria Away : {prob_vittoriaAW:.1%}\n\n\n\n\
    ')

1/1 [==============================] - 0s 41ms/step
Predizione risultato : Vittoria Away
con probabilità di Pareggio : 0.7%,
probabilità di vittoria Casa : 0.1%
e probabilità di vittoria Away : 99.3%



    


/home/omori/venv38/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
